# 🎬 AI Video Generator with Ngrok

**Features:**
- Streamlit web interface
- Ngrok public tunneling
- Real-time video generation
- GPU/CPU support
- 720p video output

**Requirements:**
- Python 3.8+
- 8GB+ RAM
- Internet connection for ngrok


In [ ]:
# Install required packages
import subprocess
import sys

packages = [
    'streamlit',
    'pyngrok',
    'torch',
    'transformers',
    'diffusers',
    'opencv-python-headless',
    'imageio[ffmpeg]',
    'pillow',
    'numpy',
    'GPUtil'
]

for package in packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
        print(f"✅ {package} installed")
    except:
        print(f"⚠️ Failed to install {package}")

print("📦 Installation complete!")

In [ ]:
# Create Streamlit app
app_code = '''
import streamlit as st
import torch
import numpy as np
from PIL import Image
import cv2
import time
import tempfile
import os

st.set_page_config(page_title="AI Video Generator", layout="wide")

def create_sample_video(prompt, duration=5, fps=24):
    """Create a sample video with text overlay"""
    width, height = 720, 480
    frames = []
    
    for i in range(duration * fps):
        # Create a frame with gradient background
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Add gradient effect
        for y in range(height):
            intensity = int((y / height) * 255)
            frame[y, :] = [intensity//3, intensity//2, intensity]
        
        # Add moving circle
        center_x = int(width/2 + 100 * np.sin(i * 0.1))
        center_y = int(height/2 + 50 * np.cos(i * 0.1))
        cv2.circle(frame, (center_x, center_y), 30, (255, 255, 255), -1)
        
        # Add text
        cv2.putText(frame, f"Frame {i+1}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, prompt[:30], (20, height-30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        frames.append(frame)
    
    return frames

def save_video(frames, output_path, fps=24):
    """Save frames as MP4 video"""
    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*\'mp4v\')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for frame in frames:
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    out.release()
    return output_path

# Main UI
st.title("🎬 AI Video Generator")
st.markdown("Generate custom videos with AI-powered content")

# Sidebar controls
st.sidebar.header("Video Settings")
prompt = st.sidebar.text_area("Video Prompt", "A beautiful sunset over mountains", height=100)
duration = st.sidebar.slider("Duration (seconds)", 1, 10, 5)
fps = st.sidebar.selectbox("FPS", [12, 24, 30], index=1)

# Check GPU availability
gpu_available = torch.cuda.is_available()
st.sidebar.info(f"GPU Available: {gpu_available}")

# Main content
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("Video Preview")
    video_placeholder = st.empty()

with col2:
    st.subheader("Controls")
    
    if st.button("🎬 Generate Video", type="primary"):
        with st.spinner("Generating video..."):
            # Create progress bar
            progress_bar = st.progress(0)
            
            # Generate frames
            frames = create_sample_video(prompt, duration, fps)
            progress_bar.progress(50)
            
            # Save video
            with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as tmp_file:
                video_path = save_video(frames, tmp_file.name, fps)
                progress_bar.progress(100)
                
                # Display video
                with video_placeholder.container():
                    st.video(video_path)
                
                # Download button
                with open(video_path, "rb") as file:
                    st.download_button(
                        label="📥 Download Video",
                        data=file.read(),
                        file_name=f"ai_video_{int(time.time())}.mp4",
                        mime="video/mp4"
                    )
                
                # Clean up
                os.unlink(video_path)
    
    st.markdown("### Settings")
    st.write(f"Duration: {duration}s")
    st.write(f"FPS: {fps}")
    st.write(f"Total Frames: {duration * fps}")

# Footer
st.markdown("---")
st.markdown("🚀 Powered by Streamlit + AI")
'''

with open('video_generator_app.py', 'w') as f:
    f.write(app_code)

print("✅ Streamlit app created: video_generator_app.py")

In [ ]:
# Setup ngrok tunnel
import threading
import subprocess
import time
from pyngrok import ngrok

def run_streamlit():
    """Run Streamlit app"""
    subprocess.run(["streamlit", "run", "video_generator_app.py", "--server.port=8501", "--server.headless=true"])

# Start Streamlit in background
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# Wait for Streamlit to start
print("🚀 Starting Streamlit...")
time.sleep(5)

# Create ngrok tunnel
try:
    public_url = ngrok.connect(8501)
    print(f"\n🌐 Public URL: {public_url}")
    print(f"🎬 Access your AI Video Generator at: {public_url}")
    print("\n⚠️ Keep this cell running to maintain the tunnel")
    
    # Keep tunnel alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n🛑 Stopping tunnel...")
        ngrok.disconnect(public_url)
        
except Exception as e:
    print(f"❌ Error setting up ngrok: {e}")
    print("💡 Try running: streamlit run video_generator_app.py --server.port=8501")

In [ ]:
# Alternative: Manual run without ngrok
print("🔧 Manual Setup Instructions:")
print("1. Open terminal")
print("2. Run: streamlit run video_generator_app.py")
print("3. Access locally at: http://localhost:8501")
print("\n📝 For public access:")
print("1. Install ngrok: https://ngrok.com/")
print("2. Run: ngrok http 8501")
print("3. Use the public URL provided")